# Learning Objectives

In this notebook, you will craft sophisticated ETL jobs that interface with a variety of common data sources, such as 
- REST APIs (HTTP endpoints)
- RDBMS
- Hive tables (managed tables)
- Various file formats (csv, json, parquet, etc.)

d

# Interview Questions

As you progress through the practice, attempt to answer the following questions:

## Columnar File
- What is a columnar file format and what advantages does it offer?
- Why is Parquet frequently used with Spark and how does it function?
- How do you read/write data from/to a Parquet file using a DataFrame?

## Partitions
- How do you save data to a file system by partitions? (Hint: Provide the code)
- How and why can partitions reduce query execution time? (Hint: Give an example)

## JDBC and RDBMS
- How do you load data from an RDBMS into Spark? (Hint: Discuss the steps and JDBC)

## REST API and HTTP Requests
- How can Spark be used to fetch data from a REST API? (Hint: Discuss making API requests)

## ETL Job One: Parquet file
### Extract
Extract data from the managed tables (e.g. `bookings_csv`, `members_csv`, and `facilities_csv`)

### Transform
Data transformation requirements https://pgexercises.com/questions/aggregates/fachoursbymonth.html

### Load
Load data into a parquet file

### What is Parquet? 

Columnar files are an important technique for optimizing Spark queries. Additionally, they are often tested in interviews.
- https://www.youtube.com/watch?v=KLFadWdomyI
- https://www.databricks.com/glossary/what-is-parquet

In [0]:
# Write your solution here
#Extract data from the managed tables.

bk = spark.sql("select * from bookings")
bk.show(5)

mem= spark.sql("SELECT* FROM members")
mem.show(5)

fac= spark.sql("select * from facilities")
fac.show(5)
     


+------+-----+-----+-------------------+-----+
|bookid|facid|memid|          starttime|slots|
+------+-----+-----+-------------------+-----+
|     0|    3|    1|2012-07-03 11:00:00|    2|
|     1|    4|    1|2012-07-03 08:00:00|    2|
|     2|    6|    0|2012-07-03 18:00:00|    2|
|     3|    7|    1|2012-07-03 19:00:00|    2|
|     4|    8|    1|2012-07-03 10:00:00|    1|
+------+-----+-----+-------------------+-----+
only showing top 5 rows

+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
|memid| surname|firstname|             address|zipcode|     telephone|recommendedby|           joindate|
+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
|    0|   GUEST|    GUEST|               GUEST|      0|(000) 000-0000|         null|2012-07-01 00:00:00|
|    1|   Smith|   Darren|8 Bloomsbury Clos...|   4321|  555-555-5555|         null|2012-07-02 12:02:05|
|    2|   Smith|    Tracy|8

## ETL Job Two: Partitions

### Extract
Extract data from the managed tables (e.g. `bookings_csv`, `members_csv`, and `facilities_csv`)

### Transform
Transform the data https://pgexercises.com/questions/joins/threejoin.html

### Load
Partition the result data by facility column and then save to `threejoin_delta` managed table. Additionally, they are often tested in interviews.

hint: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.partitionBy.html

What are paritions? 

Partitions are an important technique to optimize Spark queries
- https://www.youtube.com/watch?v=hvF7tY2-L3U&t=268s

In [0]:
#Transform data as requested into dataframe.

#Produce a list of the total number of slots booked per facility in the month of September 2012. Produce an output table consisting of facility id and slots, sorted by the number of slots.

from pyspark.sql.functions import col,sum

Slots= bk.filter((col("starttime") >= "2012-09-01") & (col("starttime") < "2012-10-01"))
#Slots.show()

totalslots= Slots.groupBy(bk.facid).agg(sum('slots').alias('TotalSlots')).orderBy('TotalSlots').show()

+-----+----------+
|facid|TotalSlots|
+-----+----------+
|    5|     122.0|
|    3|     422.0|
|    7|     426.0|
|    8|     471.0|
|    6|     540.0|
|    2|     570.0|
|    1|     588.0|
|    0|     591.0|
|    4|     648.0|
+-----+----------+



In [0]:

### Transform data as per request

#How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name followed by the facility name.

from pyspark.sql.functions import concat,concat_ws,asc,desc

li= ['Tennis Court 1','Tennis Court 2']

data = mem.join(bk, mem.memid==bk.memid, 'inner')\
.join(fac, bk.facid==fac.facid, 'inner')

data1= data.select(concat_ws(' ',mem.firstname, mem.surname).alias('Name'),fac.name.alias('facility')).filter((fac.name.isin(li)))

tennis= data1.distinct().orderBy('Name','facility')
tennis.show()


+--------------+--------------+
|          Name|      facility|
+--------------+--------------+
|    Anne Baker|Tennis Court 1|
|    Anne Baker|Tennis Court 2|
|  Burton Tracy|Tennis Court 1|
|  Burton Tracy|Tennis Court 2|
|  Charles Owen|Tennis Court 1|
|  Charles Owen|Tennis Court 2|
|  Darren Smith|Tennis Court 2|
| David Farrell|Tennis Court 1|
| David Farrell|Tennis Court 2|
|   David Jones|Tennis Court 1|
|   David Jones|Tennis Court 2|
|  David Pinker|Tennis Court 1|
| Douglas Jones|Tennis Court 1|
| Erica Crumpet|Tennis Court 1|
|Florence Bader|Tennis Court 1|
|Florence Bader|Tennis Court 2|
|   GUEST GUEST|Tennis Court 1|
|   GUEST GUEST|Tennis Court 2|
|Gerald Butters|Tennis Court 1|
|Gerald Butters|Tennis Court 2|
+--------------+--------------+
only showing top 20 rows



## ETL Job Three: HTTP Requests

### Extract
Extract daily stock price data price from the following companies, Google, Apple, Microsoft, and Tesla. 

Data Source
- API: https://rapidapi.com/alphavantage/api/alpha-vantage
- Endpoint: GET `TIME_SERIES_DAILY`

Sample HTTP request

```
curl --request GET \
	--url 'https://alpha-vantage.p.rapidapi.com/query?function=TIME_SERIES_DAILY&symbol=TSLA&outputsize=compact&datatype=json' \
	--header 'X-RapidAPI-Host: alpha-vantage.p.rapidapi.com' \
	--header 'X-RapidAPI-Key: [YOUR_KEY]'

```

Sample Python HTTP request

```
import requests

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {
    "function":"TIME_SERIES_DAILY",
    "symbol":"IBM",
    "datatype":"json",
    "outputsize":"compact"
}

headers = {
    "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com",
    "X-RapidAPI-Key": "[YOUR_KEY]"
}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()

# Now 'data' contains the daily time series data for "IBM"
```

### Transform
Find **weekly** max closing price for each company.

hints: 
  - Use a `for-loop` to get stock data for each company
  - Use the spark `union` operation to concat all data into one DF
  - create a new `week` column from the data column
  - use `group by` to calcualte max closing price

### Load
- Partition `DF` by company
- Load the DF in to a managed table called, `max_closing_price_weekly`

In [0]:
import requests
from pyspark.sql import SparkSession

# Set up your Alpha Vantage API key and base URL
api_key = "4a3c8ab6f6mshf33c91629f0ecd6p194a71jsn0810a5a1a065"
base_url = "https://alpha-vantage.p.rapidapi.com/query"

# List of companies (symbols) you want to fetch data for
#companies = ["GOOGL", "AAPL", "MSFT", "TSLA"]

# Initialize a Spark session
spark = SparkSession.builder.appName("StockPriceAnalysis").getOrCreate()

# Loop through each company and fetch data

#GOOGL
params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": "GOOGL",
        "apikey": api_key,
        "datatype":"json",
        "outputsize":"compact"
    }
headers = {
        "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com",
        "X-RapidAPI-Key": "4a3c8ab6f6mshf33c91629f0ecd6p194a71jsn0810a5a1a065"
    }
    
response1 = requests.get(base_url, params=params, headers=headers)
data1 = response1.json()

#AAPL
params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": "AAPL",
        "apikey": api_key,
        "datatype":"json",
        "outputsize":"compact"
    }
headers = {
        "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com",
        "X-RapidAPI-Key": "4a3c8ab6f6mshf33c91629f0ecd6p194a71jsn0810a5a1a065"
    }
    
response2 = requests.get(base_url, params=params, headers=headers)
data2 = response2.json()

#MSFT
params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": "MSFT",
        "apikey": api_key,
        "datatype":"json",
        "outputsize":"compact"
    }
headers = {
        "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com",
        "X-RapidAPI-Key": "4a3c8ab6f6mshf33c91629f0ecd6p194a71jsn0810a5a1a065"
    }
    
response3 = requests.get(base_url, params=params, headers=headers)
data3 = response3.json()

#TSLA
params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": "TSLA",
        "apikey": api_key,
        "datatype":"json",
        "outputsize":"compact"
    }
headers = {
        "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com",
        "X-RapidAPI-Key": "4a3c8ab6f6mshf33c91629f0ecd6p194a71jsn0810a5a1a065"
    }
    
response4 = requests.get(base_url, params=params, headers=headers)
data4 = response4.json()



In [0]:

 # Convert JSON data to a PySpark DataFrame

from pyspark.sql.functions import col, to_date, year, weekofyear, max,lit
# Create a list to store DataFrames for each company
dataframes = []
#companies = ["GOOGL", "AAPL", "MSFT", "TSLA"]

#GOOGL

stock_data1 = [(date, float(values["1. open"]), float(values["2. high"]), float(values["3. low"]), float(values["4. close"]), 
                int(values["5. volume"])) for date, values in data1["Time Series (Daily)"].items()]
columns = ["date", "open", "high", "low", "close", "volume"]
stock_df1 = spark.createDataFrame(stock_data1, columns)
    
# Convert date column to a proper date type
stock_df1 = stock_df1.withColumn("date", to_date(col("date")))
stock_df1 = stock_df1.withColumn("company", lit("GOOGL"))
dataframes.append(stock_df1)
#stock_df1.show()  

#AAPL
stock_data2 = [(date, float(values["1. open"]), float(values["2. high"]), float(values["3. low"]), float(values["4. close"]),
                 int(values["5. volume"]))for date, values in data2["Time Series (Daily)"].items()]
columns = ["date", "open", "high", "low", "close", "volume"]
stock_df2 = spark.createDataFrame(stock_data2, columns)
    
# Convert date column to a proper date type
stock_df2 = stock_df2.withColumn("date", to_date(col("date")))
stock_df2 = stock_df2.withColumn("company", lit("AAPL"))
dataframes.append(stock_df2)
#stock_df2.show() 

#MSFT
stock_data3 = [(date, float(values["1. open"]), float(values["2. high"]), float(values["3. low"]), float(values["4. close"]),
                 int(values["5. volume"]))for date, values in data3["Time Series (Daily)"].items()]
columns = ["date", "open", "high", "low", "close", "volume"]
stock_df3 = spark.createDataFrame(stock_data3, columns)
    
# Convert date column to a proper date type
stock_df3 = stock_df3.withColumn("date", to_date(col("date")))
stock_df3 = stock_df3.withColumn("company", lit("MSFT"))
dataframes.append(stock_df3)
#stock_df3.show() 


#TSLA
stock_data4 = [(date, float(values["1. open"]), float(values["2. high"]), float(values["3. low"]), float(values["4. close"]),
                 int(values["5. volume"]))for date, values in data4["Time Series (Daily)"].items()]
columns = ["date", "open", "high", "low", "close", "volume"]
stock_df4 = spark.createDataFrame(stock_data4, columns)
    
# Convert date column to a proper date type
stock_df4 = stock_df4.withColumn("date", to_date(col("date")))
stock_df4 = stock_df4.withColumn("company", lit("TSLA"))
dataframes.append(stock_df4)
#stock_df4.show() 



all_data_df = dataframes[0]
for df in dataframes[1:]:
    all_data_df = all_data_df.union(df)

all_data_df = all_data_df.withColumn("year", year(col("date")))
all_data_df = all_data_df.withColumn("week", weekofyear(col("date")))


# Group by year and week and find the maximum closing price
weekly_max_closing = all_data_df.groupBy("year", "week", "close","company").agg(max("close").alias("max_close")).orderBy("week", ascending= False)

# Show the result
weekly_max_closing.show()

+----+----+------+-------+---------+
|year|week| close|company|max_close|
+----+----+------+-------+---------+
|2023|  52|193.15|   AAPL|   193.15|
|2023|  52|193.58|   AAPL|   193.58|
|2023|  52|141.52|  GOOGL|   141.52|
|2023|  52|140.37|  GOOGL|   140.37|
|2023|  52|139.69|  GOOGL|   139.69|
|2023|  52|140.23|  GOOGL|   140.23|
|2023|  52|193.05|   AAPL|   193.05|
|2023|  52|192.53|   AAPL|   192.53|
|2023|  52|375.28|   MSFT|   375.28|
|2023|  52|376.04|   MSFT|   376.04|
|2023|  52|374.07|   MSFT|   374.07|
|2023|  52|374.66|   MSFT|   374.66|
|2023|  52|256.61|   TSLA|   256.61|
|2023|  52|253.18|   TSLA|   253.18|
|2023|  52|248.48|   TSLA|   248.48|
|2023|  52|261.44|   TSLA|   261.44|
|2023|  51|136.65|  GOOGL|   136.65|
|2023|  51|138.34|  GOOGL|   138.34|
|2023|  51|141.49|  GOOGL|   141.49|
|2023|  51|140.42|  GOOGL|   140.42|
+----+----+------+-------+---------+
only showing top 20 rows



In [0]:
weekly_max_closing.write.partitionBy("company").mode('overwrite').parquet("/FileStore/tables/weekly_max_closing.parquet")
spark.read.parquet("/FileStore/tables/weekly_max_closing.parquet/company=AAPL").show()

+----+----+------+---------+
|year|week| close|max_close|
+----+----+------+---------+
|2023|  52|193.05|   193.05|
|2023|  52|192.53|   192.53|
|2023|  52|193.58|   193.58|
|2023|  52|193.15|   193.15|
|2023|  51|195.89|   195.89|
|2023|  51|196.94|   196.94|
|2023|  51|194.83|   194.83|
|2023|  51| 193.6|    193.6|
|2023|  51|194.68|   194.68|
|2023|  50|198.11|   198.11|
|2023|  50|197.96|   197.96|
|2023|  50|197.57|   197.57|
|2023|  50|193.18|   193.18|
|2023|  50|194.71|   194.71|
|2023|  49|195.71|   195.71|
|2023|  49|192.32|   192.32|
|2023|  49|194.27|   194.27|
|2023|  49|193.42|   193.42|
|2023|  49|189.43|   189.43|
|2023|  48|189.37|   189.37|
+----+----+------+---------+
only showing top 20 rows



In [0]:
spark.read.parquet("/FileStore/tables/weekly_max_closing.parquet/company=GOOGL").show()

+----+----+------+---------+
|year|week| close|max_close|
+----+----+------+---------+
|2023|  52|141.52|   141.52|
|2023|  52|140.37|   140.37|
|2023|  52|139.69|   139.69|
|2023|  52|140.23|   140.23|
|2023|  51|140.42|   140.42|
|2023|  51| 135.8|    135.8|
|2023|  51|136.65|   136.65|
|2023|  51|138.34|   138.34|
|2023|  51|141.49|   141.49|
|2023|  50| 132.6|    132.6|
|2023|  50|131.94|   131.94|
|2023|  50|132.57|   132.57|
|2023|  50|133.29|   133.29|
|2023|  50|132.52|   132.52|
|2023|  49|136.93|   136.93|
|2023|  49|134.99|   134.99|
|2023|  49|130.02|   130.02|
|2023|  49|130.99|   130.99|
|2023|  49|129.27|   129.27|
|2023|  48|131.86|   131.86|
+----+----+------+---------+
only showing top 20 rows



In [0]:

spark.read.parquet("/FileStore/tables/weekly_max_closing.parquet/company=TSLA").show()

+----+----+------+---------+
|year|week| close|max_close|
+----+----+------+---------+
|2023|  52|256.61|   256.61|
|2023|  52|253.18|   253.18|
|2023|  52|248.48|   248.48|
|2023|  52|261.44|   261.44|
|2023|  51|252.54|   252.54|
|2023|  51| 254.5|    254.5|
|2023|  51|257.22|   257.22|
|2023|  51|247.14|   247.14|
|2023|  51|252.08|   252.08|
|2023|  50|239.29|   239.29|
|2023|  50|251.05|   251.05|
|2023|  50| 253.5|    253.5|
|2023|  50|239.74|   239.74|
|2023|  50|237.01|   237.01|
|2023|  49|238.72|   238.72|
|2023|  49|243.84|   243.84|
|2023|  49|239.37|   239.37|
|2023|  49|235.58|   235.58|
|2023|  49|242.64|   242.64|
|2023|  48|244.14|   244.14|
+----+----+------+---------+
only showing top 20 rows



## ETL Job Four: RDBMS


### Extract
Extract RNA data from a public PostgreSQL database.

- https://rnacentral.org/help/public-database
- Extract 100 RNA records from the `rna` table (hint: use `limit` in your sql)
- hint: use `spark.read.jdbc` https://docs.databricks.com/external-data/jdbc.html

### Transform
We want to load the data as it so there is no transformation required.


### Load
Load the DF in to a managed table called, `rna_100_records`

In [0]:
#Extract RNA data from a public PostgreSQL database. only 100 rows 

rna_100_records = (spark.read
  .format("jdbc")
  .option("url", "jdbc:postgresql://hh-pgsql-public.ebi.ac.uk:5432/pfmegrnargs")
  .option("dbtable", "rna")
  .option("user", "reader")
  .option("password", "NWDMCE5xdipIjRrp")
  .load()
  .limit(100)
)

rna_100_records.count()
rna_100_records.printSchema()



root
 |-- id: long (nullable = true)
 |-- upi: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- userstamp: string (nullable = true)
 |-- crc64: string (nullable = true)
 |-- len: integer (nullable = true)
 |-- seq_short: string (nullable = true)
 |-- seq_long: string (nullable = true)
 |-- md5: string (nullable = true)



In [0]:

#fetching all rows 
rna_records = (spark.read
  .format("jdbc")
  .option("url", "jdbc:postgresql://hh-pgsql-public.ebi.ac.uk:5432/pfmegrnargs")
  .option("dbtable", "rna")
  .option("user", "reader")
  .option("password", "NWDMCE5xdipIjRrp")
  .load()
)

rna_records.count()

In [0]:
#Load the DF in to a managed table called, rna_100_records

#drop table if exist 

#spark.sql("DROP TABLE IF EXISTS rna_100_record")
rna_100_records.write.saveAsTable("rna_100_records_1")